# 多实例安装mysql 5.6.50

## 1. 卸载旧软件、关防火墙、安装依赖
### 卸载旧软件
[root@localhost local]# rpm -qa | grep mysql

[root@localhost local]# rpm -qa | grep mariadb

            mariadb-libs-5.5.56-2.el7.x86_64
            
[root@localhost local]# yum -y remove mariadb-libs.x86_64 autoconf

### 设置防火墙
[root@localhost agr]# vim /etc/selinux/config
                
                #This file controls the state of SELinux on the system.
                #SELINUX= can take one of these three values:
                #    enforcing - SELinux security policy is enforced.
                #    permissive - SELinux prints warnings instead of enforcing.
                #    disabled - No SELinux policy is loaded.
                SELINUX=permissive
                #SELINUXTYPE= can take one of three values:
                #   targeted - Targeted processes are protected,
                #   minimum - Modification of targeted policy. Only selected processes are protected. 
                #   mls - Multi Level Security protection.
                SELINUXTYPE=targeted
[root@localhost agr]# shutdown -r now   #重启           
[root@localhost agr]# getenforce
                
                Permissive

### 安装依赖

[root@localhost agr]# yum install libaio


## 2.下载二进制格式的mysql

[root@localhost agr]# wget https://mirrors.tuna.tsinghua.edu.cn/mysql/downloads/MySQL-5.6/mysql-5.6.50-linux-glibc2.12-x86_64.tar.gz


: 

## 3.解压文件，设置软连接
[root@localhost agr]# cd /usr/local

[root@localhost agr]# tar -zxvf mysql-5.6/mysql-5.6.50-linux-glibc2.12-x86_64.tar.gz

[root@localhost agr]# ls /usr/local/

bin  games    lib    libexec                              sbin   src
etc  include  lib64 mysql-5.6/mysql-5.6.50-linux-glibc2.12-x86_64.tar.gz  share

[root@localhost agr]# cd /usr/local/

### 软连接
[root@localhost local]# ln -sv mysql-5.7.38-linux-glibc2.12-x86_64 mysql 





## 4.创建MySQL用户
[root@localhost agr]# useradd -M -r -s /sbin/nologin mysql
            
            #/sbin/nologin是不允许登录
            
[root@localhost agr]# id mysql

uid=995(mysql) gid=992(mysql) groups=992(mysql)

### 登录用户显示bash-4.2的解决方法

[root@localhost agr]cp -r/etc/skel/.bash* /home/mysql

### “This account is currently not available”的解决方法

[root@localhost agr] vim /etc/passwd
                
                #“/sbin /nologin”改成“/bin/bash”
[root@localhost agr] cat /etc/passwd

                ...
               mysql:x:987:980::/home/mysql:/bin/bash


## 5.多用户文件夹
[root@localhost agr]# yum install -y tree

[root@localhost agr]# mkdir -p /mysql/{3306,3307,3308}/{data,my.cnf}

[root@localhost agr]# vim /mysql/3307/my.cnf

[mysql]

[client]
port=3307
socket=/mysql/3307/temp/mysql_3307.sock


[mysqld]
user=mysql
port=3307

socket=/mysql/3307/temp/mysql_3307.sock
basedir=/usr/local/mysql/
datadir=/mysql/3307/data

innodb_buffer_pool_size=128M #缓存InnoDB的容量
character-set-server=utf8

log-bin=/mysql/3307/log/mysql-bin
server-id = 3

[mysqld_safe]
log-error=/mysql/3307/log/mysql_3307_error.log
pid-file=/mysql/3307/mysqld_3307.pid
     



### 验证
[root@localhost 3307]# cat my.cnf
[mysql]

[client]
port=3307
socket=/mysql/3307/temp/mysql_3307.sock


[mysqld]
user=mysql
port=3307

socket=/mysql/3307/temp/mysql_3307.sock
basedir=/usr/local/mysql/
datadir=/mysql/3307/data

innodb_buffer_pool_size=128M #缓存InnoDB的容量
character-set-server=utf8

log-bin=/mysql/3307/log/mysql-bin
server-id = 3

[mysqld_safe]
log-error=/mysql/3307/log/mysql_3307_error.log
pid-file=/mysql/3307/mysqld_3307.pid

[root@localhost 3307]# 




## 6.修改文件权限
 [root@localhost agr]# chown -R mysql.mysql /mysql/
 
 [root@localhost agr]# ls -l /mysql
 
                total 0
                drwxr-xr-x. 3 mysql mysql 32 2月  26 15:48 3306
                drwxr-xr-x. 3 mysql mysql 32 2月  26 15:50 3307
                drwxr-xr-x. 3 mysql mysql 32 2月  26 15:50 3308

 

## 7.添加环境变量

[root@localhost 3307]# mysql
ERROR 2002 (HY000): Can't connect to local MySQL server through socket '/tmp/mysql.sock' (2)



[root@localhost agr]# vim /etc/profile

#文件最后添加 export PATH=/usr/local/mysql/bin/:$PATH

### 刷新缓存
[root@localhost agr]# source /etc/profile
  
### 验证  
[root@localhost 3307]# which mysql
/usr/local/mysql/bin/mysql
               
                
### 方式二

[root@localhost 3307]#tail -1 /etc/profile
export PATH=/usr/local/mysql/bin/:$PATH

             


## 8.配置启动脚本



[root@localhost 3306]# cat mysql_3307

[mysqld]
port=3307
mysql_user="mysql"
CmdPath="/usr/local/mysql/bin"
mysql_sock="/mysql/${port}/temp/mysql_${port}.sock"
mysqld_pid_file_path=/mysql/${port}/mysqld_${port}.pid
function_start_mysql()
{
if [ ! -e "$mysql_sock" ]; then
    printf "Starting MySQL... \n"
    /bin/sh ${CmdPath}/mysqld_safe --defaults-file=/mysql/${port}/my.cnf --pid-file=$mysqld_pid_file_path 2>&1 >/dev/null  &
    sleep 3
else
    printf "MySQL is running...\n"
    exit 1
fi
}
function_stop_mysql()
{
if  [ ! -e  "$mysql_sock" ];  then 
      printf "MySQL is stopped ...\n"
      exit 1
else
      printf "Stoping MYSQL ...\n"
      mysqld_pid = 'cat "mysqld_pid_file_path"'
if(kill -0 $mysqld_pid 2>/dev/null)
	then
	kill $mysqld_pid
	sleep 2
	fi
fi
}

function_restart_mysql()
{
	printf "Restarting MySQL...\n"
	function_stop_mysql
	sleep 2
	function_start_mysql
}

case "$1" in
start)
	function_start_mysql
;;
stop)
	function_stop_mysql
;;
restart)
 	function_restart_mysql
;;
*)
	printf "Usage: /mysql/${port}/mysql{start|stop|restart}\n"
esac



### 赋予执行权限
chmod +x /mysql/3307/mysql_3307

### 降低文件夹权限
chown -R mysql.mysql /mysql/





[root@localhost profile.d]# /usr/local/mysql/scripts/mysql_install_db --basedir=/usr/local/mysql --datadir=/mysql/3306/data --user=mysql


[root@youxi1 mysql]# cp support-files/mysql.server /etc/init.d/mysqld

[root@youxi1 mysql]# chmod +x /etc/init.d/mysqld

## 9.初始化

[root@localhost 3307]# /usr/local/mysql/scripts/mysql_install_db --defaults-file=/mysql/3307/my.cnf --basedir=/usr/local/mysql/ --datadir=/mysql/3307/data/ --user=mysql


### 验证及sock登录

[root@localhost 3308]# netstat -tunlp|grep mysql
tcp6       0      0 :::3307            :::*              LISTEN      4189/mysqld        
tcp6       0      0 :::3308            :::*              LISTEN      4569/mysqld   



[root@localhost 3308]# mysql -S /mysql/3307/temp/mysql_3307.sock
Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 1
Server version: 5.6.50-log MySQL Community Server (GPL)

Copyright (c) 2000, 2020, Oracle and/or its affiliates. All rights reserved.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> 
